# Pill Classification using mobile net 

In [53]:
## library  as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import re
from PIL import Image
import shutil
import xml.etree.ElementTree as et
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

## library import
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import tensorflow.keras as keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns


import numpy as np
import pandas as pd
import os
import re
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt


### Dataset preparation

In [10]:
current_directory = os.getcwd()
print("현재 경로:", current_directory)

현재 경로: /Users/seyeong/workspace/drugmaster_pillClassification/PillClassification


In [67]:
data_dir = 'test_augmented_images'
train = 'train'
validation = 'validation'
test = 'test'

aug_data_dir = os.path.join(data_dir)
print(aug_data_dir)
image_dir = os.path.join(train, 'rectangle_img')
print(image_dir)
bbox_dir = os.path.join(train, 'rectangle_annotation')
print(bbox_dir)

val_img_dir = os.path.join(validation, 'rectangle_img')
val_bbox_dir = os.path.join(validation, 'rectangle_annotation')

test_img_dir = os.path.join(test, 'rectangle_img')
test_bbox_dir = os.path.join(test, 'rectangle_annotation')

test_augmented_images
train/rectangle_img
train/rectangle_annotation


In [68]:
# train set split
for file in os.listdir(aug_data_dir):
    file_format = re.split('[.]',file)[-1]
    if file_format=='jpg':
        shutil.copy(os.path.join(aug_data_dir, file), os.path.join(image_dir, file))
    else:
        shutil.copy(os.path.join(aug_data_dir, file), os.path.join(bbox_dir, file))

# validation set split
for file in os.listdir('validation_raw'):
    file_format = re.split('[.]',file)[-1]
    if file_format=='jpg':
        shutil.copy(os.path.join('validation_raw', file), os.path.join(val_img_dir, file))
    else:
        shutil.copy(os.path.join('validation_raw', file), os.path.join(val_bbox_dir, file))

# test set split
for file in os.listdir('test_raw'):
    file_format = re.split('[.]',file)[-1]
    if file_format=='jpg':
        shutil.copy(os.path.join('test_raw', file), os.path.join(test_img_dir, file))
    else:
        shutil.copy(os.path.join('test_raw', file), os.path.join(test_bbox_dir, file))

In [69]:
## image file 수 확인
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']
print(len(image_files))
image_files[:5]

40000


['ApurtranTablet150mg(Irbesartan)_0_augmented_image_333.jpg',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_1_augmented_image_519.jpg',
 'GasridTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_826.jpg',
 'GasbetTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_480.jpg',
 'NumentaminSustainedReleaseCapsule8mg(GalantamineBromide)_0_augmented_image_678.jpg']

In [31]:
# class수 확인
class_list = set()
for image_file in image_files:
    file_name = re.split('_',image_file)[0]
    class_list.add(file_name)

class_list = list(class_list)
print(len(class_list))

20


In [32]:
class_list.sort()
class_list[:5]

['ApurtranTablet150mg(Irbesartan)',
 'BalsanTablet80mg(Valsartan)',
 'BalthrepTablet160mg(Valsartan)',
 'DiosartanTablet160mg(Valsartan)',
 'EscitalTablet5mg(EscitalopramOxalate)']

In [34]:
class2idx = {cls:idx for idx, cls in enumerate(class_list)}
class2idx['ApurtranTablet150mg(Irbesartan)']

0

In [39]:
train_images = os.listdir(image_dir)
val_images = os.listdir(validation)

print(len(train_images), len(val_images))

40000 16000


In [40]:
train_images[:10]

['ApurtranTablet150mg(Irbesartan)_0_augmented_image_333.jpg',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_1_augmented_image_519.jpg',
 'GasridTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_826.jpg',
 'GasbetTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_480.jpg',
 'NumentaminSustainedReleaseCapsule8mg(GalantamineBromide)_0_augmented_image_678.jpg',
 'SurosinDTablet(TamsulosinHydrochloride)_1_augmented_image_433.jpg',
 'BalsanTablet80mg(Valsartan)_0_augmented_image_473.jpg',
 'ApurtranTablet150mg(Irbesartan)_0_augmented_image_73.jpg',
 'EscitalTablet5mg(EscitalopramOxalate)_1_augmented_image_355.jpg',
 'ValsartelTablet80mg(Valsartan)_1_augmented_image_631.jpg']

In [41]:
val_images[:10]

['BalthrepTablet160mg(Valsartan)_0_augmented_image_87.txt',
 'ApurtranTablet150mg(Irbesartan)_0_augmented_image_73.jpg',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_0_augmented_image_7.jpg',
 'RosorodTablet10mg(RosuvastatinCalcium)_1_augmented_image_37.jpg',
 'ZolpidemSustainedReleaseTablet(ZolpidemTartrate)_1_augmented_image_157.jpg',
 'GasdialTablet50mg(DimethiconeMagnesium)_1_augmented_image_177.jpg',
 'GasprenTablet(MosaprideCitrateDihydrate)_1_augmented_image_73.txt',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_1_augmented_image_143.txt',
 'FeratracTablet2.5mg(Letrozole)_1_augmented_image_151.jpg',
 'SarvaltanTablet160mg(Valsartan)_0_augmented_image_130.jpg']

In [55]:
train_dir = os.path.join('train/rectangle_img')
val_dir = os.path.join('validation')

### TFRecord

In [42]:
IMG_SIZE = 100

In [43]:

## TFRecord 저장할 directory와 file 경로 설정
tfr_dir = os.path.join('tfrecord')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'cls_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'cls_val.tfr')

In [44]:
import tensorflow as tf
## TFRecord writer 생성
writer_train = tf.io.TFRecordWriter(tfr_train_dir)
writer_val = tf.io.TFRecordWriter(tfr_val_dir)

In [46]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.
# 데이터 직렬화

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [60]:
n_train = 0

train_files = os.listdir(train_dir)
for train_file in train_files:
  train_path = os.path.join(train_dir, train_file)
  image = Image.open(train_path)
  image = image.resize((IMG_SIZE, IMG_SIZE))
  bimage = image.tobytes()

  file_name = os.path.splitext(train_file)[0] #Bangal_101
  
  class_name = re.split('_',file_name)[0]
  class_num = class2idx[class_name]
  print(class_name)
  
  example = tf.train.Example(features=tf.train.Features(feature={
      'image': _bytes_feature(bimage),
      'cls_num': _int64_feature(class_num)
  }))
  writer_train.write(example.SerializeToString())
  n_train += 1

writer_train.close()
print(n_train)

ApurtranTablet150mg(Irbesartan)
LipinonTablet80mg(AtorvastatinCalciumTrihydrate)
GasridTablet5mg(MosaprideCitrateHydrate)
GasbetTablet5mg(MosaprideCitrateHydrate)
NumentaminSustainedReleaseCapsule8mg(GalantamineBromide)
SurosinDTablet(TamsulosinHydrochloride)
BalsanTablet80mg(Valsartan)
ApurtranTablet150mg(Irbesartan)
EscitalTablet5mg(EscitalopramOxalate)
ValsartelTablet80mg(Valsartan)
ValsartanTablet(Valsartan)
FeratracTablet2.5mg(Letrozole)
SarvaltanTablet160mg(Valsartan)
BalsanTablet80mg(Valsartan)
ExbanTablet80mg(Valsartan)
DiosartanTablet160mg(Valsartan)
LipinonTablet80mg(AtorvastatinCalciumTrihydrate)
ValsartelTablet160mg(Valsartan)
GasprenTablet(MosaprideCitrateDihydrate)
GasdialTablet50mg(DimethiconeMagnesium)
GasbetTablet5mg(MosaprideCitrateHydrate)
RosorodTablet10mg(RosuvastatinCalcium)
ZolpidemSustainedReleaseTablet(ZolpidemTartrate)
GasbetTablet5mg(MosaprideCitrateHydrate)
GasdialTablet50mg(DimethiconeMagnesium)
ZolpidemSustainedReleaseTablet(ZolpidemTartrate)
ValsartelTabl

In [61]:
n_val = 0

val_files = os.listdir(val_dir)
for val_file in val_files:
  val_path = os.path.join(val_dir, val_file)
  image = Image.open(val_path)
  image = image.resize((IMG_SIZE, IMG_SIZE))
  bimage = image.tobytes()

  file_name = os.path.splitext(val_file)[0] #Bangal_101
  class_name = re.split('_',file_name)[0]
  class_num = class2idx[class_name]

  example = tf.train.Example(features=tf.train.Features(feature={
      'image': _bytes_feature(bimage),
      'cls_num': _int64_feature(class_num)
  }))
  writer_val.write(example.SerializeToString())
  n_val += 1

writer_val.close()
print(n_val)

UnidentifiedImageError: cannot identify image file '/Users/seyeong/workspace/drugmaster_pillClassification/PillClassification/validation/BalthrepTablet160mg(Valsartan)_0_augmented_image_87.txt'